In [1]:
#from modelscope.hub.snapshot_download import snapshot_download

#model_dir = snapshot_download('damo/multi-modal_clip-vit-large-patch14_336_zh',
#                              cache_dir='/media/checkpoint/multi-modal_clip-vit-large-patch14_336_zh',
#                              revision='v1.0.1')

2023-09-11 02:17:37,613 - modelscope - INFO - PyTorch version 2.0.1 Found.
2023-09-11 02:17:37,615 - modelscope - INFO - TensorFlow version 2.10.0 Found.
2023-09-11 02:17:37,615 - modelscope - INFO - Loading ast index from /home/dmeck/.cache/modelscope/ast_indexer
2023-09-11 02:17:37,616 - modelscope - INFO - No valid ast index found from /home/dmeck/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-09-11 02:17:37,719 - modelscope - INFO - Loading done! Current index file version is 1.9.0, with md5 e74c3ad0d32f5b2c017304f8c2d8a5f8 and a total number of 921 components indexed
2023-09-11 02:17:38,019 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4.87k/4.87k [00:00<00:00, 490kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 126k/126k [00:00<

In [1]:
# require modelscope>=0.3.7，目前默认已经超过，您检查一下即可
# 按照更新镜像的方法处理或者下面的方法
# pip install --upgrade modelscope -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
# 需要单独安装decord，安装方法：pip install decord
import torch
from modelscope.utils.constant import Tasks
from modelscope.pipelines import pipeline
from modelscope.preprocessors.image import load_image

pipeline = pipeline(task=Tasks.multi_modal_embedding,
    model='/media/checkpoint/multi-modal_clip-vit-large-patch14_336_zh/damo/multi-modal_clip-vit-large-patch14_336_zh', model_revision='v1.0.1')



2023-09-11 22:48:17,583 - modelscope - INFO - PyTorch version 2.0.1 Found.
2023-09-11 22:48:17,586 - modelscope - INFO - TensorFlow version 2.10.0 Found.
2023-09-11 22:48:17,586 - modelscope - INFO - Loading ast index from /home/dmeck/.cache/modelscope/ast_indexer
2023-09-11 22:48:17,661 - modelscope - INFO - Loading done! Current index file version is 1.9.0, with md5 e74c3ad0d32f5b2c017304f8c2d8a5f8 and a total number of 921 components indexed
/media/gpt4-pdf-chatbot-langchain/pyenv-video/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-11 22:48:19.108475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in othe

In [2]:
import os

'''
获取输入文件夹内的所有jpg文件，并返回文件名全称列表
'''
def load_jpg(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.jpg':
                filename=os.path.join(root, file)
                L.append(filename)
        return L 
        
def batch(iterable, size):
    # range对象的step是size
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

In [15]:
data_folder = '/media/checkpoint/speech_data/抖音作品/ieAeWyXU/keyframe-features'

jpg_files=load_jpg(data_folder) 
print("获取数据，成功{}".format(len(jpg_files)))


获取数据，成功1599


In [16]:
import pickle
from tqdm import tqdm  # Import the tqdm function or class
import numpy as np
 
#对每一个文件进行操作
batch_len=3
batch_list = batch(jpg_files,batch_len)
b_unit = tqdm(enumerate(jpg_files),total=len(jpg_files))

img_embedding_list = []

for batch_file in batch_list:  
    table_data=[]
    for filename in batch_file:
        
        input_img = load_image(filename)
        # 支持一张图片(PIL.Image)或多张图片(List[PIL.Image])输入，输出归一化特征向量
        # 2D Tensor, [图片数, 特征维度]
        img_embedding = pipeline.forward({'img': input_img})['img_embedding']
        #torch.Size([1, 768])形状转换 torch.Size([768])
        tensor_squeezed = img_embedding.squeeze()
        img_embedding_list.append(tensor_squeezed)
    # 更新进度
    b_unit.update(batch_len) 


# 循环结束后关闭进度条
b_unit.close()

save_name = os.path.join(data_folder, f"features.pkl")
# Open the file in write mode
with open(save_name, 'wb') as file:
    
    features = [tensor.cpu() for tensor in img_embedding_list]
    features_np = [tensor.numpy() for tensor in features] 
    print(np.asarray(features_np).shape)
    pickle.dump(features_np, file)
    file.close
 


# Read inference data from local
with open(save_name, 'rb') as file:
    features = pickle.load(file)


features = np.asarray(features)

print(features.shape)
  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1599/1599 [00:33<00:00, 47.19it/s]


(1599, 768)
(1599, 768)
